In [2]:
import findspark
findspark.init()
import datetime
import requests
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql import SparkSession


spark = (SparkSession.builder
      .master("local[1]")
      .appName("ChallengeDatas.com")
      .getOrCreate())

requisicao = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios")
dfCities = requisicao.json()
dataRequest = dfCities
df = spark.createDataFrame(dataRequest)
df.createOrReplaceTempView("Cities")


API_KEY = "*******************************"

data = []
contador = 0

for _ in range(len(dataRequest)):
      municipio = dataRequest[contador]['nome']
      link = f"https://api.openweathermap.org/data/2.5/forecast?q={municipio}&appid={API_KEY}&lang=pt_br"
      requisicaoApi = requests.get(link)
      requisicaoApiDic = requisicaoApi.json()
      

      requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Rain", "Sim")
      requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Clear", "Não")
      requisicaoApiDic['list'][0]['weather'][0]['main'] = requisicaoApiDic['list'][0]['weather'][0]['main'].replace("Clouds", "Não")

      cidade = requisicaoApiDic['city']['name']
      codigoCidade = requisicaoApiDic['city']['id']
      dataPrevisao = requisicaoApiDic['list'][0]['dt_txt']
      regiao = ''
      pais = requisicaoApiDic['city']['country']
      lat = requisicaoApiDic['city']['coord']['lat']
      long = requisicaoApiDic['city']['coord']['lon']
      tempMax = requisicaoApiDic['list'][0]['main']['temp_max'] - 273.15
      tempMin = requisicaoApiDic['list'][0]['main']['temp_min'] - 273.15
      temperaturaMedia = (requisicaoApiDic['list'][0]['main']['temp_max'] - 273.15 + requisicaoApiDic['list'][0]['main']['temp_min'] - 273.15) / 2
      tempMed = temperaturaMedia
      vaiChover = requisicaoApiDic['list'][0]['weather'][0]['main']
      chanceDeChuva = ''
      condicaoDoTempo = requisicaoApiDic['list'][0]['weather'][0]['description']
      nascerDoSol = requisicaoApiDic['city']['sunrise']
      nascerDoSol = datetime.datetime.fromtimestamp(nascerDoSol).strftime('%H:%M:%S')
      porDoSOl = requisicaoApiDic['city']['sunset']
      porDoSOl = datetime.datetime.fromtimestamp(porDoSOl).strftime('%H:%M:%S')

      velocidadeDoVento = requisicaoApiDic['list'][0]['wind']['speed']
      
      
      data.append([cidade, codigoCidade, dataPrevisao, regiao, pais, lat, long, tempMax, tempMin, tempMed, vaiChover, 
      chanceDeChuva, condicaoDoTempo, nascerDoSol, porDoSOl, velocidadeDoVento])

      contador += 1

dataForecast = data

schema = (StructType([
      StructField('Cidade', StringType(), True),
      StructField('CodigoDaCidade', StringType(), True),
      StructField('Data', StringType(), True),
      StructField('Regiao', StringType(), True),
      StructField('Pais', StringType(), True),
      StructField('Latitude', StringType(), True),
      StructField('Longitude', StringType(), True),
      StructField('TemperaturaMaxima', StringType(), True),
      StructField('TemperaturaMinima', StringType(), True),
      StructField('TemperaturaMedia', StringType(), True),
      StructField('VaiChover', StringType(), True),
      StructField('ChanceDeChuva', StringType(), True),
      StructField('CondicaoDoTempo', StringType(), True),
      StructField('NascerDoSol', StringType(), True),
      StructField('PorDoSol', StringType(), True),
      StructField('VelocidadeMaximaDoVento', StringType(), True),
]))


dfData = spark.createDataFrame(data=dataForecast, schema=schema)
dfData.createOrReplaceTempView("Forecast")
spark.sql("select VaiChover from Forecast").show()




Commit Repository
